**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath('C:/Users/Florian Bettini/Anaconda3/ffmpeg-20200220-56df829-win64-static/bin/')
import skvideo.io
from time import time

import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Input

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

**Answer:**
* The function **act**, if in train mode, will return a random action with an espilon probability.
* Otherwise, this function will return the best action it has learnt with the **learned_act** function.
* epsilon is essential in order for the algorithm to learn. Without a random exploration, it won't be reinforcement learning.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [33]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite('Videos/'+ str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [34]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**Answer:**
- The array **position** is used to indicate where the rat is at each step of the algorithm
- The array **board** is used to indicate where bonus and malus are placed. Bonus and malus are randomly disposed on the board when we reset the environment. When the rat moves, the reward (a bonus or a malus) is set to 0, meaning the rat has eaten cheese or poison.




## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [35]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [36]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        #---------------------------------------------------------------------------#
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        #---------------------------------------------------------------------------#

        # Update stats
        score = score + win-lose
        
        # I slightly modified the code to only print around 10 results (eg if we have 100 epochs, only 10 will be printed)
        # I also decided to draw only 10 files (as it is very time consuming) and to print the execution time
        if (e+1)%(max(epochs//10,1)) == 0:
            print("Epoch {:03d} / {:03d} | Win/lose count {:.1f}/{:.1f} | Average score ({:.2f}) | time {:.2f} secs"
                  .format(e+1, epochs, win, lose, score/(1+e), time()-t))
            # Save as a mp4
            env.draw(prefix+str(e+1))
            
    print('Final score: '+str(score/epochs))

In [37]:
# define a variable to compute execution time
t = time()

# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('Videos/random10.mp4'))

Epoch 010 / 100 | Win/lose count 16.0/21.0 | Average score (-4.55) | time 5.38 secs
Epoch 020 / 100 | Win/lose count 12.5/18.0 | Average score (-3.83) | time 12.97 secs
Epoch 030 / 100 | Win/lose count 9.5/18.0 | Average score (-3.98) | time 21.37 secs
Epoch 040 / 100 | Win/lose count 8.5/13.0 | Average score (-4.20) | time 28.19 secs
Epoch 050 / 100 | Win/lose count 4.5/12.0 | Average score (-4.74) | time 35.07 secs
Epoch 060 / 100 | Win/lose count 7.5/11.0 | Average score (-4.76) | time 42.78 secs
Epoch 070 / 100 | Win/lose count 7.5/12.0 | Average score (-4.75) | time 50.82 secs
Epoch 080 / 100 | Win/lose count 3.5/8.0 | Average score (-4.64) | time 60.54 secs
Epoch 090 / 100 | Win/lose count 13.0/10.0 | Average score (-4.53) | time 68.15 secs
Epoch 100 / 100 | Win/lose count 11.0/19.0 | Average score (-4.56) | time 75.53 secs
Final score: -4.56


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer:**

1) With $T=\infty$, we have:

\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>
\end{equation*}
\begin{equation*}
= r(s, a) + E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>
\end{equation*}
\begin{equation*}
= r(s, a) + \gamma E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>
\end{equation*}


by using $\mathbb{E}(X) = \displaystyle \sum_i \mathbb{E}(X|A_i) \mathbb{P}(A_i)$ we get:

\begin{equation*}
Q^\pi(s,a) = r(s, a) + \gamma \times  \sum_{(a',s')} {E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a,s_{1}=s',a_{1}=a']} \times P(s_{1}=s',a_{1}=a' |s_{0}=s,a_{0}=a) \>
\end{equation*}

\begin{equation*}
= r(s, a) + \gamma \times  \sum_{(a',s')} {E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']} \times P(s_{1}=s',a_{1}=a' |s_{0}=s,a_{0}=a) \>
\end{equation*}

\begin{equation*}
= r(s, a) + \gamma \times  \sum_{(a',s')} Q^\pi(s',a') \times P(s_{1}=s',a_{1}=a' |s_{0}=s,a_{0}=a) \>
\end{equation*}

\begin{equation*}
= r(s, a) + \gamma \times E(s',a') ~ P(.|s,a) [Q^\pi(s',a')] \>
\end{equation*}

Hence, with linearity:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}] \>(s',a')]
\end{equation*}

2) We have:

\begin{equation*}
Q^{*}(s,a)= \max_{\pi}{Q^{\pi}(s,a)} = \max_{\pi}{E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]}
\end{equation*}

\begin{equation*}
= r(s,a) + \gamma \times \max_{\pi}{E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]}
\end{equation*}


Now, let $\pi^*$ (we assume it exists) be deterministic such as $Q^*=Q^{\pi^*}$. Then:

\begin{equation*}
Q^{\pi^*}(s',a')=\max_{a''} Q^{\pi^*}(s', a'')
\end{equation*}

The following inequality holds:

\begin{equation*}
\max_{\pi}{E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]} \geq E_{(s',a')\sim \pi^*(.|s,a)}[Q^*(s',a')]
\end{equation*}

On the other hand:

\begin{equation*}
\max_{\pi}{E_{(s',a')\sim \pi^*(.|s,a)}[Q^{\pi}(s',a')]} = \max_{\pi}E_{s'\sim \pi^*(.|s,a)}[\max_{a''}{Q^*(s',a'')]}
\end{equation*}

\begin{equation*}
\leq \max_{a''} E_{s'\sim \pi^*(.|s,a)}[Q^*(s',a'')]
\end{equation*}

\begin{equation*}
\leq \max_{a''} \max_{\pi} E_{(s',a')\sim p^{\pi}(.|s,a)}[Q^{\pi}(s',a')]
\end{equation*}

\begin{equation*}
\leq \max_{\pi} E_{(s',a')\sim p^{\pi}(.|s,a)}[Q^{\pi}(s',a')]
\end{equation*}

Hence:

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

3) With the linearity, we have :

\begin{equation*}
E_{s' \sim p(.|s,a)}\left[r(s, a) + \gamma \max_{a'} Q^* (s', a') - Q^*(s,a)\right] = 0
\end{equation*}

By minimizing the squared norm of this quantity, we get the following loss function:  

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [16]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        # I slightly modified the code to only print around 10 results (eg if we have 100 epochs, only 10 will be printed)
        # I also decided to plot the execution time
        if (e+1)%(max(epoch//10,1)) == 0:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | time {:.2f} mins"
                  .format(e+1, epoch, loss, win, lose, win-lose, (time()-t)/60))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [17]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))
        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, 0)))
                
        # Clipping the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(n_hidden_neurons, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [19]:
from time import time
t, n_hidden_neurons = time(), 20
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('Videos/fc_train' + str(epochs_train) + '.mp4'))

Epoch 010/100 | Loss 0.0076 | Win/lose count 4.5/4.0 (0.5) | time 1.12 mins
Epoch 020/100 | Loss 0.0005 | Win/lose count 4.5/0 (4.5) | time 2.20 mins
Epoch 030/100 | Loss 0.0072 | Win/lose count 4.5/5.0 (-0.5) | time 3.22 mins
Epoch 040/100 | Loss 0.0036 | Win/lose count 4.5/4.0 (0.5) | time 4.43 mins
Epoch 050/100 | Loss 0.0068 | Win/lose count 3.5/1.0 (2.5) | time 5.38 mins
Epoch 060/100 | Loss 0.0027 | Win/lose count 5.0/2.0 (3.0) | time 6.41 mins
Epoch 070/100 | Loss 0.0032 | Win/lose count 5.5/4.0 (1.5) | time 7.43 mins
Epoch 080/100 | Loss 0.0009 | Win/lose count 7.0/2.0 (5.0) | time 8.42 mins
Epoch 090/100 | Loss 0.0040 | Win/lose count 11.5/7.0 (4.5) | time 9.40 mins
Epoch 100/100 | Loss 0.0128 | Win/lose count 3.0/2.0 (1.0) | time 10.43 mins


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [20]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(n_hidden_neurons, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(n_hidden_neurons*2, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [21]:
t, n_hidden_neurons = time(), 32
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('Videos/cnn_train' + str(epochs_train) + '.mp4'))

Epoch 010/100 | Loss 0.0089 | Win/lose count 7.0/3.0 (4.0) | time 1.02 mins
Epoch 020/100 | Loss 0.0019 | Win/lose count 4.0/1.0 (3.0) | time 2.13 mins
Epoch 030/100 | Loss 0.0071 | Win/lose count 5.5/3.0 (2.5) | time 3.18 mins
Epoch 040/100 | Loss 0.0096 | Win/lose count 6.0/2.0 (4.0) | time 4.22 mins
Epoch 050/100 | Loss 0.0106 | Win/lose count 9.5/3.0 (6.5) | time 5.26 mins
Epoch 060/100 | Loss 0.0180 | Win/lose count 12.5/2.0 (10.5) | time 6.27 mins
Epoch 070/100 | Loss 0.0201 | Win/lose count 8.5/0 (8.5) | time 7.28 mins
Epoch 080/100 | Loss 0.0208 | Win/lose count 22.0/7.0 (15.0) | time 8.31 mins
Epoch 090/100 | Loss 0.0261 | Win/lose count 21.5/4.0 (17.5) | time 9.34 mins
Epoch 100/100 | Loss 0.0164 | Win/lose count 18.5/3.0 (15.5) | time 10.37 mins


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
t, n_hidden_neurons = time(), 32
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
t, n_hidden_neurons = time(), 20
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Epoch 010 / 100 | Win/lose count 3.0/0.0 | Average score (2.75) | time 7.03 secs
Epoch 020 / 100 | Win/lose count 0.5/0.0 | Average score (2.92) | time 18.86 secs
Epoch 030 / 100 | Win/lose count 1.0/0.0 | Average score (2.55) | time 28.38 secs
Epoch 040 / 100 | Win/lose count 4.5/0.0 | Average score (2.67) | time 37.57 secs
Epoch 050 / 100 | Win/lose count 6.0/1.0 | Average score (2.66) | time 45.90 secs
Epoch 060 / 100 | Win/lose count 5.0/1.0 | Average score (2.67) | time 54.12 secs
Epoch 070 / 100 | Win/lose count 6.5/1.0 | Average score (2.79) | time 62.38 secs
Epoch 080 / 100 | Win/lose count 4.5/1.0 | Average score (2.81) | time 70.63 secs
Epoch 090 / 100 | Win/lose count 1.5/0.0 | Average score (2.73) | time 79.06 secs
Epoch 100 / 100 | Win/lose count 4.5/0.0 | Average score (2.76) | time 87.50 secs
Final score: 2.76
Test of the FC
Epoch 010 / 100 | Win/lose count 2.0/0.0 | Average score (0.80) | time 7.12 secs
Epoch 020 / 100 | Win/lose count 0.5/2.0 | Average 

In [23]:
HTML(display_videos('Videos/cnn_test' + str(epochs_test) + '.mp4'))

In [24]:
HTML(display_videos('Videos/fc_test' + str(epochs_test) + '.mp4'))

In [25]:
# train with different number of neurons in the hidden layer
for n_hidden_neurons in [10, 30, 20]:
    print('')
    print('FC | number of hidden neurons: %d' % n_hidden_neurons)
    t = time()
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent_fc, env, epochs_train, prefix='fc_train')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')

for n_hidden_neurons in [16, 64, 32]:
    print('')
    print('CNN | number of hidden neurons: %d' % n_hidden_neurons)
    t = time()
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent_cnn,env,epochs_train,prefix='cnn_train')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')


FC | number of hidden neurons: 10
Epoch 010/100 | Loss 0.0043 | Win/lose count 8.5/5.0 (3.5) | time 1.11 mins
Epoch 020/100 | Loss 0.0007 | Win/lose count 4.0/3.0 (1.0) | time 2.21 mins
Epoch 030/100 | Loss 0.0198 | Win/lose count 2.0/2.0 (0.0) | time 3.32 mins
Epoch 040/100 | Loss 0.0175 | Win/lose count 5.5/4.0 (1.5) | time 4.36 mins
Epoch 050/100 | Loss 0.0016 | Win/lose count 4.0/3.0 (1.0) | time 5.42 mins
Epoch 060/100 | Loss 0.0046 | Win/lose count 3.0/0 (3.0) | time 6.51 mins
Epoch 070/100 | Loss 0.0037 | Win/lose count 5.0/4.0 (1.0) | time 7.57 mins
Epoch 080/100 | Loss 0.0110 | Win/lose count 5.0/2.0 (3.0) | time 8.57 mins
Epoch 090/100 | Loss 0.0132 | Win/lose count 2.0/3.0 (-1.0) | time 9.55 mins
Epoch 100/100 | Loss 0.0121 | Win/lose count 3.0/2.0 (1.0) | time 10.57 mins
Test of the FC
Epoch 010 / 100 | Win/lose count 2.5/2.0 | Average score (0.75) | time 641.43 secs
Epoch 020 / 100 | Win/lose count 0.0/0.0 | Average score (0.93) | time 649.61 secs
Epoch 030 / 100 | Win/lo

Epoch 100 / 100 | Win/lose count 8.0/0.0 | Average score (5.73) | time 767.05 secs
Final score: 5.73

CNN | number of hidden neurons: 32
Epoch 010/100 | Loss 0.0004 | Win/lose count 3.5/3.0 (0.5) | time 1.06 mins
Epoch 020/100 | Loss 0.0043 | Win/lose count 4.5/4.0 (0.5) | time 2.08 mins
Epoch 030/100 | Loss 0.0011 | Win/lose count 4.0/1.0 (3.0) | time 3.28 mins
Epoch 040/100 | Loss 0.0104 | Win/lose count 8.0/2.0 (6.0) | time 4.38 mins
Epoch 050/100 | Loss 0.0078 | Win/lose count 5.5/4.0 (1.5) | time 5.46 mins
Epoch 060/100 | Loss 0.0107 | Win/lose count 2.0/4.0 (-2.0) | time 6.49 mins
Epoch 070/100 | Loss 0.0071 | Win/lose count 12.0/2.0 (10.0) | time 7.54 mins
Epoch 080/100 | Loss 0.0024 | Win/lose count 6.0/1.0 (5.0) | time 8.59 mins
Epoch 090/100 | Loss 0.0074 | Win/lose count 10.5/3.0 (7.5) | time 9.54 mins
Epoch 100/100 | Loss 0.0069 | Win/lose count 7.5/4.0 (3.5) | time 10.50 mins
Test of the CNN
Epoch 010 / 100 | Win/lose count 8.0/1.0 | Average score (4.85) | time 636.87 secs

**Answer**

* After testing different parameters (eg, the number of neurons in the hidden layer), best architectures are:
    * 20 neurons in the hidden layer for the fully-connected network
    * 64 output filters after the first convolution for the CNN network (the number of output filters is then multipled by 2 in the second convolution layer)

* The main issue is that our rat gets stuck at some point. As a consequence, it doesn't explore the entire map, which gives bad results. We will tackle this issue in question 10.

In [26]:
# testing for different temperatures
for temperature in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    
    print('')
    print("Testing for a temperature of %0.1f" % temperature)
    env = Environment(grid_size=size, max_time=T,temperature=temperature)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN')
    t, n_hidden_neurons = time(), 32
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    t, n_hidden_neurons = time(), 20
    test(agent_fc,env,epochs_test,prefix='fc_test')


Testing for a temperature of 0.1
Test of the CNN
Epoch 010 / 100 | Win/lose count 0.5/0.0 | Average score (0.70) | time 9.26 secs
Epoch 020 / 100 | Win/lose count 3.0/1.0 | Average score (1.18) | time 17.51 secs
Epoch 030 / 100 | Win/lose count 0.0/0.0 | Average score (1.17) | time 25.70 secs
Epoch 040 / 100 | Win/lose count 2.0/0.0 | Average score (1.26) | time 34.38 secs
Epoch 050 / 100 | Win/lose count 1.5/0.0 | Average score (1.31) | time 42.83 secs
Epoch 060 / 100 | Win/lose count 0.5/0.0 | Average score (1.17) | time 52.56 secs
Epoch 070 / 100 | Win/lose count 1.0/0.0 | Average score (1.16) | time 61.94 secs
Epoch 080 / 100 | Win/lose count 0.5/0.0 | Average score (1.14) | time 71.39 secs
Epoch 090 / 100 | Win/lose count 1.0/0.0 | Average score (1.14) | time 80.61 secs
Epoch 100 / 100 | Win/lose count 0.0/0.0 | Average score (1.14) | time 89.86 secs
Final score: 1.14
Test of the FC
Epoch 010 / 100 | Win/lose count 0.0/1.0 | Average score (-0.10) | time 9.48 secs
Epoch 020 / 100 

Epoch 050 / 100 | Win/lose count 0.0/0.0 | Average score (-0.01) | time 40.98 secs
Epoch 060 / 100 | Win/lose count 0.0/0.0 | Average score (0.04) | time 50.07 secs
Epoch 070 / 100 | Win/lose count 0.0/0.0 | Average score (0.11) | time 60.26 secs
Epoch 080 / 100 | Win/lose count 3.0/2.0 | Average score (0.14) | time 69.57 secs
Epoch 090 / 100 | Win/lose count 0.5/2.0 | Average score (0.09) | time 80.59 secs
Epoch 100 / 100 | Win/lose count 1.0/0.0 | Average score (0.09) | time 89.25 secs
Final score: 0.09

Testing for a temperature of 0.6
Test of the CNN
Epoch 010 / 100 | Win/lose count 21.0/2.0 | Average score (23.00) | time 7.01 secs
Epoch 020 / 100 | Win/lose count 36.5/5.0 | Average score (22.57) | time 16.22 secs
Epoch 030 / 100 | Win/lose count 36.5/9.0 | Average score (21.55) | time 24.73 secs
Epoch 040 / 100 | Win/lose count 39.5/4.0 | Average score (23.02) | time 34.09 secs
Epoch 050 / 100 | Win/lose count 38.0/5.0 | Average score (23.31) | time 44.91 secs
Epoch 060 / 100 | Wi

Epoch 080 / 100 | Win/lose count 14.5/0.0 | Average score (45.04) | time 64.79 secs
Epoch 090 / 100 | Win/lose count 79.0/0.0 | Average score (45.56) | time 73.05 secs
Epoch 100 / 100 | Win/lose count 16.5/0.0 | Average score (45.35) | time 81.17 secs
Final score: 45.355
Test of the FC
Epoch 010 / 100 | Win/lose count 4.0/0.0 | Average score (2.50) | time 6.74 secs
Epoch 020 / 100 | Win/lose count 0.5/0.0 | Average score (3.12) | time 14.80 secs
Epoch 030 / 100 | Win/lose count 2.5/0.0 | Average score (3.10) | time 23.54 secs
Epoch 040 / 100 | Win/lose count 1.5/0.0 | Average score (2.95) | time 32.36 secs
Epoch 050 / 100 | Win/lose count 5.5/0.0 | Average score (3.02) | time 40.85 secs
Epoch 060 / 100 | Win/lose count 0.5/0.0 | Average score (3.14) | time 49.16 secs
Epoch 070 / 100 | Win/lose count 5.5/0.0 | Average score (3.16) | time 57.31 secs
Epoch 080 / 100 | Win/lose count 5.0/0.0 | Average score (3.20) | time 65.46 secs
Epoch 090 / 100 | Win/lose count 0.5/0.0 | Average score (

**Answer**

One can observe (see below) that the score increases with temperature. This phenomenon is due to the increasing amount a bonuses (on the map) when the temperature increases. In addition, the more bonus there are, the more likely a bonus and a malus can be created on a same square: hence, the malus is deleted and there are more bonus on the map, making the life easier for our rat !

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [30]:
def train_explore(agent,env,epoch,prefix='',epsilon_decay=0.99):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True) # given code

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Decrease the exploration rate
        agent.set_epsilon(agent.epsilon * epsilon_decay)
        
        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw('Videos/'+prefix+str(e+1))
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.2f}) | time {:.2f} mins"
              .format(e+1, epoch, loss, win, lose, win-lose, (time()-t)/60))

        # Update stats
        score += win-lose

        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        self.x = 0
        self.y = 1
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0]    = 256
        b[self.board<0, 2] = 256
        b[self.x,self.y,:]   = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:]  = 0
        b[:,:2,:]  = 0
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 
        self.position[:, -2:] = -1
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # ---------- #
        # given code
        reward = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y] 
        
        # ---------- #
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # given code
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0
        self.board = bonus + malus
        
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # given code
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [40]:
# Training
t, n_hidden_neurons = time(), 64
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('Videos/cnn_train_explore' + str(epochs_train) + '.mp4'))

Epoch 010/100 | Loss 0.0204 | Win/lose count 21.5/23.7 (-2.20) | time 1.12 mins
Epoch 020/100 | Loss 0.0376 | Win/lose count 24.5/18.0 (6.50) | time 2.19 mins
Epoch 030/100 | Loss 0.0428 | Win/lose count 19.5/20.8 (-1.30) | time 3.24 mins
Epoch 040/100 | Loss 0.0403 | Win/lose count 21.0/17.2 (3.80) | time 4.29 mins
Epoch 050/100 | Loss 0.0474 | Win/lose count 21.0/18.6 (2.40) | time 5.34 mins
Epoch 060/100 | Loss 0.0417 | Win/lose count 20.5/22.6 (-2.10) | time 6.38 mins
Epoch 070/100 | Loss 0.0150 | Win/lose count 22.0/19.0 (3.00) | time 7.42 mins
Epoch 080/100 | Loss 0.0591 | Win/lose count 23.5/19.9 (3.60) | time 8.47 mins
Epoch 090/100 | Loss 0.0407 | Win/lose count 24.0/16.9 (7.10) | time 9.51 mins
Epoch 100/100 | Loss 0.0925 | Win/lose count 21.5/20.6 (0.90) | time 10.59 mins


In [43]:
# Evaluation
t, n_hidden_neurons = time(), 64
test(agent, env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore' + str(epochs_test) + '.mp4'))

Epoch 010 / 100 | Win/lose count 24.0/6.0 | Average score (18.05) | time 7.04 secs
Epoch 020 / 100 | Win/lose count 21.0/8.0 | Average score (17.10) | time 15.34 secs
Epoch 030 / 100 | Win/lose count 20.5/3.0 | Average score (17.40) | time 23.69 secs
Epoch 040 / 100 | Win/lose count 21.5/6.0 | Average score (17.81) | time 32.11 secs
Epoch 050 / 100 | Win/lose count 20.5/8.0 | Average score (18.16) | time 40.52 secs
Epoch 060 / 100 | Win/lose count 27.5/3.0 | Average score (18.35) | time 49.24 secs
Epoch 070 / 100 | Win/lose count 24.0/3.0 | Average score (18.44) | time 60.00 secs
Epoch 080 / 100 | Win/lose count 24.0/8.0 | Average score (18.38) | time 68.71 secs
Epoch 090 / 100 | Win/lose count 27.0/4.0 | Average score (18.23) | time 77.14 secs
Epoch 100 / 100 | Win/lose count 24.0/4.0 | Average score (18.44) | time 85.75 secs
Final score: 18.44


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***